In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as dsets 
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.utils.data as Data
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torch.nn.functional as F

In [2]:
df = pd.read_table('Train_Dst_Auction_ZScore_CF_9.txt',header=None,delim_whitespace=True).T

In [3]:
# parameter for small cnn
input_size = 144
conv_kernel_1 = 21
padding_1 = 10
channels=4
conv_kernel_2 = 21
padding_2 = 10
pool_kernel = 3
num_classes = 3
num_epochs = 7
batch_size = 64
learning_rate = 0.0001
torch.manual_seed(0)

In [4]:
# input_size = 144
# num_classes = 3
# num_epochs = 20
# batch_size = 128
# learning_rate = 0.001
torch.manual_seed(0)
class MyDataset(Dataset):
    def __init__(self, factorReturn, stockReturn):
        data = []
        factorReturn = np.array(factorReturn)
        stockReturn = np.array(stockReturn)
        for i in range(len(factorReturn)):
            data.append((factorReturn[i],stockReturn[i]-1))
        self.data = data 
    def __getitem__(self, index):
        fn, label = self.data[index]
        return fn, label
    def __len__(self):
        return len(self.data)

class Net(nn.Module):
    def __init__(self, input_size, conv_kernel_1=21, padding_1=10, channels=4, conv_kernel_2=5, padding_2=2, pool_kernel=4, num_classes=3):
        super(Net, self).__init__()
        self.C1 = nn.Conv1d(1, channels, conv_kernel_1, 1, padding_1)
        self.R1 = nn.ReLU()
        self.S2 = nn.MaxPool1d(kernel_size=pool_kernel,stride=1)
        self.C3 = nn.Conv1d(channels, channels, conv_kernel_2, 1, padding_2)
        self.R2 = nn.ReLU()
        self.S4 = nn.MaxPool1d(kernel_size=pool_kernel,stride=1)
        self.F6 = nn.Linear(in_features=560, out_features=84)
        self.R4 = nn.ReLU()
        self.outlayer = nn.Linear(84 * 1 * 1, num_classes)
    
    def forward(self, x):
        
        x = self.C1(x)
        x = self.R1(x)
        x = self.S2(x)
        x = self.C3(x)
        x = self.R2(x)
        x = self.S4(x)
        x = x.view(x.size(0), -1)
        x = self.F6(x)
        x = self.R4(x)
#         import pdb
#         pdb.set_t
        x = self.outlayer(x)
        return x

In [44]:
class Net1(nn.Module):
    def __init__(self, input_size, conv_kernel_1, padding_1, channels, conv_kernel_2, padding_2, pool_kernel, num_classes):
        super(Net, self).__init__()
        self.C1 = nn.Conv1d(1, 8, 21, 1, 10)
        self.R1 = nn.ReLU()
        self.S1 = nn.MaxPool1d(kernel_size=pool_kernel,stride=2)
        
        self.C2 = nn.Conv1d(8, 8, 21, 1, 10)
        self.R2 = nn.ReLU()
        self.S2 = nn.MaxPool1d(kernel_size=pool_kernel,stride=2)
        
        self.C3 = nn.Conv1d(8, 8, 21, 1, 10)
        self.R3 = nn.ReLU()
        self.S3 = nn.MaxPool1d(kernel_size=pool_kernel,stride=2)
        
        self.C4 = nn.Conv1d(8, 8, 11, 1, 5)
        self.R4 = nn.ReLU()
        self.S4 = nn.MaxPool1d(kernel_size=pool_kernel,stride=1)
        
        self.C5 = nn.Conv1d(8, 8, 5, 1, 2)
        self.R5 = nn.ReLU()
        self.S5 = nn.MaxPool1d(kernel_size=pool_kernel,stride=1)
        
        self.F6 = nn.Linear(in_features=104, out_features=64)
        self.R6 = nn.ReLU()
        
        self.F7 = nn.Linear(in_features=64, out_features=17)
        self.R7 = nn.ReLU()
        
        self.outlayer = nn.Linear(17 * 1 * 1, 3)
    
    def forward(self, x):
        
        x = self.S1(self.R1(self.C1(x)))
        x = self.S2(self.R2(self.C2(x)))
        x = self.S3(self.R3(self.C3(x)))
        x = self.S4(self.R4(self.C4(x)))
        x = self.S5(self.R5(self.C5(x)))
        x = x.view(x.size(0), -1)
        x = self.R6(self.F6(x))
        x = self.R7(self.F7(x))
        x = self.outlayer(x)
        return x

In [5]:
def cNN1Predict(input_size, conv_kernel_1, padding_1, channels, conv_kernel_2, padding_2, pool_kernel, num_classes,trainX,trainY,testX,testY,idx,batch_size,num_epochs,validation=True):
    if validation == False:
        trainX = np.concatenate((trainX,testX),axis=0)
        trainY = np.concatenate((trainY,testY),axis=0)
    data = MyDataset(trainX,trainY)
    train_loader = torch.utils.data.DataLoader(dataset=data,
                                               batch_size=batch_size,
                                               shuffle=True)
    del data
    
    
    data_test = MyDataset(testX,testY)
    test_loader = torch.utils.data.DataLoader(dataset=data_test,
                                               batch_size=batch_size,
                                               shuffle=True)
    del data_test

    net = Net(input_size, conv_kernel_1, padding_1, channels, conv_kernel_2, padding_2, pool_kernel, num_classes).cuda()
    criterion = nn.CrossEntropyLoss() 
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, weight_decay=0.05)
    for epoch in range(num_epochs):
        for i, (feature, labels) in enumerate(train_loader):
            feature = Variable(feature.view(-1, input_size))
            labels = Variable(labels)
            feature = feature.to(torch.float32).cuda().unsqueeze(1)
            labels = labels.to(torch.float32).cuda()
            optimizer.zero_grad(set_to_none=True)
            outputs = net(feature)
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()

    result = pd.DataFrame()
    label = pd.DataFrame()
    for feature, labels in test_loader:
        feature = Variable(feature.view(-1, input_size))
        feature = feature.to(torch.float32).cuda().unsqueeze(1)
        outputs = net(feature)
        _, predicted = torch.max(outputs, 1)
        result = pd.concat([result,pd.DataFrame(predicted.cpu().detach().numpy())])
        label = pd.concat([label,pd.DataFrame(labels)])
    
    result.columns = ['pred']
    result['label'] = label[0]
    result.index = idx
    return net, result

In [6]:
def testSetResult(net, testX, testY, idx):
    data = MyDataset(testX, testY)
    test_loader = torch.utils.data.DataLoader(dataset=data,
                                               batch_size=batch_size,
                                               shuffle=True)
    result = pd.DataFrame()
    for feature, labels in test_loader:
        feature = Variable(feature.view(-1, input_size))
        feature = feature.to(torch.float32).cuda().unsqueeze(1)
        outputs = net(feature)
        _, predicted = torch.max(outputs, 1)
        result = pd.concat([result,pd.DataFrame(predicted.cpu().detach().numpy())])
    
    result.columns = ['pred']
    result.index = idx
    return result

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [17]:
testSet = pd.read_table('Test_Dst_Auction_ZScore_CF_9.txt',header=None,delim_whitespace=True).T
def trainForLabel(labelOrder,testSet):
    i = 144+labelOrder
    trainX,validX = train_test_split(df.iloc[:,:144])
    trainY,validY = df.loc[trainX.index][i],df.loc[validX.index][i]
    
    idx = validY.index
    net, result = cNN1Predict(input_size, conv_kernel_1, padding_1, channels, conv_kernel_2, padding_2, pool_kernel, num_classes,np.array(trainX),np.array(trainY),np.array(validX),np.array(validY),idx,batch_size=batch_size,num_epochs=num_epochs,validation=False)

    test_Result = testSetResult(net, testSet.iloc[:,:144], testSet[i], idx=testSet.index)
    test_Result['label'] = testSet[i]-1
    average='macro'
    accuracy = accuracy_score(test_Result['label'], test_Result['pred'])
    recall = recall_score(test_Result['label'], test_Result['pred'], average=average)
    f1 = f1_score(test_Result['label'], test_Result['pred'], average=average)
    precise = precision_score(test_Result['label'], test_Result['pred'], average=average)
    re = pd.DataFrame([accuracy,precise,recall,f1],index=['accuracy','precise','recall','f1'],columns=[str(i)])
    return re

In [18]:
cm = pd.DataFrame()
for i in range(5):
    re = trainForLabel(i,testSet)
    cm = pd.concat([cm,re],axis=1)
# cm.to_csv('cm_small.csv')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [84]:
cm

,144,145,146,147,148
accuracy,0.712859,0.595919,0.517369,0.414336,0.365664
precise,0.554848,0.450582,0.393647,0.341862,0.251048
recall,0.712859,0.595919,0.517369,0.414336,0.365664
f1,0.598559,0.470266,0.382958,0.285484,0.274566


In [85]:
cm_large

,144,145,146,147,148
accuracy,0.716644,0.611114,0.532859,0.426416,0.374094
precise,0.513579,0.373460,0.283939,0.181831,0.139946
recall,0.716644,0.611114,0.532859,0.426416,0.374094
f1,0.598352,0.463605,0.370470,0.254948,0.203692


In [78]:
cm_large = pd.DataFrame()
for i in range(5):
    re = trainForLabel(i,testSet)
    cm_large = pd.concat([cm_large,re],axis=1)
cm_large.to_csv('cm_large.csv')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedM

In [9]:
# parameter for small cnn
input_size = 144
conv_kernel_1 = 21
padding_1 = 10
channels=4
conv_kernel_2 = 21
padding_2 = 10
pool_kernel = 3
num_classes = 3
num_epochs = 7
batch_size = 32
learning_rate = 0.001
torch.manual_seed(0)

In [18]:
## tuning process

i = 144+3
trainX,validX = train_test_split(df.iloc[:,:144])
trainY,validY = df.loc[trainX.index][i],df.loc[validX.index][i]

idx = validY.index
net, result = cNN1Predict(input_size, conv_kernel_1, padding_1, channels, conv_kernel_2, padding_2, pool_kernel, num_classes,np.array(trainX),np.array(trainY),np.array(validX),np.array(validY),idx,batch_size=batch_size,num_epochs=num_epochs)
# result['label'] = validY-1
len(result[result['pred']==result['label']].dropna())/len(validY)

0.3984926676740893

In [56]:
i = 144+0
trainX,validX = train_test_split(df.iloc[:,:i])
trainY,validY = df.loc[trainX.index][i],df.loc[validX.index][i]

idx = validY.index
net, result = cNN1Predict(input_size, conv_kernel_1, padding_1, channels, conv_kernel_2, padding_2, pool_kernel, num_classes,np.array(trainX),np.array(trainY),np.array(validX),np.array(validY),idx,batch_size=batch_size,num_epochs=num_epochs)
result['label'] = validY-1
len(result[result['pred']==result['label']].dropna())/len(validY)

0.6845911860025281